# Feature Engineering

1. Select low/high-variance variables
2. Dropping redundant variables
3. RFE

In [21]:
import pandas as pd
import dslabs_functions as dslabs
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from matplotlib.pyplot import subplots, show
from matplotlib.pyplot import savefig, figure

## tratar da primeira coluna nos notebooks anteriores
winning_alternative_scaling = 2
folder = "feat_eng_csvs"
filename = f'../data_preparation_csvs/scaling_csvs/data_scaled_minmax_alt{winning_alternative_scaling}.csv'
data_scaling: pd.DataFrame = pd.read_csv(filename, sep=',', decimal='.', na_values='')
print(f"Dataset nr records={data_scaling.shape[0]}", f"nr variables={data_scaling.shape[1]}")
filename_prefix = "../data_preparation_csvs/outliers_treatment_csvs/feature_eng_"

Dataset nr records=83816 nr variables=32


In [22]:
# Save to file
def save(df, name):
    df.to_csv(f'{filename_prefix}{name}_data.csv', index=False)

    ## Separate into Train and Test and save in different files
    train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

    ## Save the training and testing sets to CSV files
    train_data.to_csv(f'../data_preparation_csvs/{folder}/train_data_mv_{name}.csv', index=False)
    test_data.to_csv(f'../data_preparation_csvs/{folder}/test_data_mv_{name}.csv', index=False)

    ## Evaluate Approach
    file_tag = "Credit_Score"
    target = "Credit_Score"

    figure()
    eval: dict[str, list] = dslabs.evaluate_approach(train_data, test_data, target=target, metric="recall")
    dslabs.plot_multibar_chart(
        ["NB", "KNN"], eval, title=f"{file_tag} evaluation", percentage=True
    )
    savefig(f"../data_preparation_images/{folder}_result/data_mv_{name}_eval.png")
    show()

In [23]:
target = "stroke"
file_tag = "stroke"
#print(data_scaling.describe().loc["std"]*data_scaling.describe().loc["std"])

print(data_scaling["Credit-Builder Loan"])

print("Original variables", data_scaling.columns.to_list())
vars2drop: list[str] = dslabs.select_low_variance_variables(data_scaling, 3, target=target)
print("Variables to drop", vars2drop)

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
83811    0.0
83812    0.0
83813    0.0
83814    0.0
83815    0.0
Name: Credit-Builder Loan, Length: 83816, dtype: float64
Original variables ['Month', 'Occupation', 'Payment_of_Min_Amount', 'CreditMix', 'Payment_Behaviour', 'Payday Loan', 'Personal Loan', 'Debt Consolidation Loan', 'Auto Loan', 'Not Specified Loan', 'Student Loan', 'Credit-Builder Loan', 'Mortgage Loan', 'Home Equity Loan', 'Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'NumofLoan', 'Delay_from_due_date', 'NumofDelayedPayment', 'ChangedCreditLimit', 'NumCreditInquiries', 'OutstandingDebt', 'CreditUtilizationRatio', 'Credit_History_Age', 'TotalEMIpermonth', 'Amountinvestedmonthly', 'MonthlyBalance', 'Credit_Score']
Variables to drop ['Month', 'Occupation', 'Payment_of_Min_Amount', 'CreditMix', 'Payment_Behaviour', 'Payday Loan', 'Personal Loan', 'Debt Consolidation Loan', 'Auto Loan', 

In [ ]:
vars2drop: list[str] = dslabs.select_low_variance_variables(
    train, max_threshold=1.2, target=target
)
train_cp, test_cp = apply_feature_selection(
    train, test, vars2drop, filename=f"data/{file_tag}", tag="lowvar"
)
print(f"Original data: train={train.shape}, test={test.shape}")
print(f"After low variance FS: train_cp={train_cp.shape}, test_cp={test_cp.shape}")